Joel Gritter & Kindeep Singh Kargil

COSC 4P80 - Seminar Demo

March 29, 2021

In [ ]:
# Music output
!sudo apt-get install fluidsynth
!pip install midi2audio
!pip install mingus
from mingus.containers import Note, NoteContainer, Track
from mingus.midi.midi_file_out import write_NoteContainer, write_Track
from midi2audio import FluidSynth

fsy = FluidSynth()

# imports for data manipulation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# imports for machine learning
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 30 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt5x11extras5 amd64 5.9.5-0ubuntu1 [8,596 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/uni

In [ ]:
# read in the notes, make an array with 0's, except for the current note
def read_and_format(input_filepath):
  input_data = []
  with open(input_filepath) as input_file:
    for line in input_file:
      values = line.split(",")
      for value in values:
        tmp = [0.0] * 88
        v = int(value)
        tmp[v-1] = 1.0
        input_data.append(tmp)
  
  return input_data
  

input_data = read_and_format("k330-allegro-moderato.csv")

In [ ]:
# get the previous 20 notes, predict the next note
def generate_datasets(input_array, n_prev = 20):
  temp_x = [input_array[i:i+n_prev] for i in range(len(input_array) - n_prev)]
  temp_y = [input_array[i+n_prev] for i in range(len(input_array) - n_prev)]

  return np.array(temp_x), np.array(temp_y)

x, y = generate_datasets(input_data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle=False)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(y_train[0])

(2008, 20, 88) (2008, 88)
(502, 20, 88) (502, 88)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# build the model itself
model = Sequential()
model.add(LSTM(30))
model.add(Dense(88, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# train the model
model.fit(x_train, y_train, batch_size=10, epochs=100, validation_split=0.05)

Epoch 1/100
191/191 [==============================] - 6s 16ms/step - loss: 3.7611 - accuracy: 0.1142 - val_loss: 2.4947 - val_accuracy: 0.1683
Epoch 2/100
191/191 [==============================] - 2s 11ms/step - loss: 2.7096 - accuracy: 0.1582 - val_loss: 2.4281 - val_accuracy: 0.1683
Epoch 3/100
191/191 [==============================] - 2s 10ms/step - loss: 2.6589 - accuracy: 0.1553 - val_loss: 2.4447 - val_accuracy: 0.1683
Epoch 4/100
191/191 [==============================] - 2s 11ms/step - loss: 2.6416 - accuracy: 0.1686 - val_loss: 2.3916 - val_accuracy: 0.2178
Epoch 5/100
191/191 [==============================] - 2s 11ms/step - loss: 2.5794 - accuracy: 0.1843 - val_loss: 2.3179 - val_accuracy: 0.2871
Epoch 6/100
191/191 [==============================] - 2s 11ms/step - loss: 2.4923 - accuracy: 0.2081 - val_loss: 2.2978 - val_accuracy: 0.3564
Epoch 7/100
191/191 [==============================] - 2s 11ms/step - loss: 2.3799 - accuracy: 0.2602 - val_loss: 2.2254 - val_accuracy:

In [ ]:
# test the model
model.evaluate(x_test, y_test)

16/16 [==============================] - 0s 4ms/step - loss: 0.7386 - accuracy: 0.7888


[0.7385919094085693, 0.788844645023346]

In [ ]:
# See incorrectly predicted 
predictions = model.predict(x_test)
incorrect_indices = []
for (index, (prediction, target)) in enumerate(zip(predictions, y_test)):
  pred = np.argmax(prediction)
  tar = np.argmax(target)
  if pred != tar:
    incorrect_indices.append(index)

print(", ".join(map(str, incorrect_indices)))

7, 16, 51, 78, 80, 84, 87, 104, 130, 133, 139, 147, 148, 152, 157, 162, 173, 179, 187, 200, 201, 202, 203, 204, 206, 215, 217, 218, 219, 220, 222, 225, 226, 238, 241, 246, 250, 253, 255, 256, 258, 259, 260, 262, 267, 269, 270, 272, 275, 276, 278, 308, 313, 316, 318, 324, 326, 332, 337, 338, 344, 345, 350, 352, 356, 357, 360, 361, 374, 379, 382, 384, 390, 392, 396, 399, 402, 411, 423, 424, 425, 430, 435, 439, 440, 442, 443, 445, 448, 453, 454, 459, 460, 465, 467, 469, 471, 476, 479, 481, 483, 486, 490, 497, 498, 500


In [ ]:
# Predict song
test_in = x_test[0]
test_out = y_test[0]
# initial - provide inital 20 notes
# n - how many predicted notes to add (i.e. expand by this number)
def make_big_song(initial, n):
  res =[ x for x in initial]
  for _ in range(n):
    next = model.predict(np.array([res[-20:],]))[0]
    res.append(next)

  return np.array(res)

test = make_big_song(test_in, 60)
print(test.shape)

(80, 88)


In [ ]:
# Expects n x 88
def vector_to_midi(arr, filename="nice.midi"):
  track = Track()
  for note_arr in arr:
    note_num = int(np.argmax(note_arr))
    note = Note()
    note.from_int(note_num - 3)
    track.add_notes(note)
  write_Track(filename, track)
  print("Done!")
  
vector_to_midi(test)

Done!


In [ ]:
def predict_to_file(first_20_notes, expected, filename="nice"):
  next = model.predict(np.array([first_20_notes]))
  actual_next = np.array([expected])
  next_file = filename + "_predicted_note"
  actual_next_file = filename + "_actual_note"
  orig_file = filename + "_first_20_notes"

  vector_to_midi(next, next_file + ".midi")
  vector_to_midi(actual_next, actual_next_file + ".midi")
  vector_to_midi(first_20_notes, orig_file + ".midi")

  # This conversion not seem to work
  # fsy.midi_to_audio(next_file + ".midi", next_file + ".mp3")
  # fsy.midi_to_audio(actual_next_file + ".midi", actual_next_file + ".mp3")
  # fsy.midi_to_audio(orig_file + ".midi",  orig_file + ".mp3")

predict_to_file(test_in, test_out)

inci = incorrect_indices[0]
predict_to_file(x_test[inci], y_test[inci], 'first_incorrect')


Done!
Done!
Done!
Done!
Done!
Done!
